
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)



# 6 Corpus Exploration


## Text Mining for Historians (with Python)
## A Gentle Introduction to Working with Textual Data in Python

### Created by Kaspar Beelen and Luke Blaxill

### For the German Historical Institute, London

<img align="left" src="https://www.ghil.ac.uk/typo3conf/ext/wacon_ghil/Resources/Public/Images/institute_icon_small.png">




This Notebook explores various tools for analysing and comparing texts at the corpus level. As such, these are your first ventures into "macro-analysis" with Python. The methods described here are particularly powerful in combination with the techniques for content selection explained in Notebook 5 **Corpus Creation**.

More specifically, we will have a closer look at:

- **Keyword in Context Analysis**: Explore context of words, similar to concordance in AntConc
- **Collocations**: Compute which tokens tend to co-occur together
- **Feature selection**: Compute which tokens are distinctive for a subset of texts

## 6.1 Keyword in Context

Computers are excellent in indexing, organizing and retrieving information. However, interpreting information (especially natural language) is still a difficult task. Keyword-in-Context (KWIC) analysis, brings together the best of both worlds: the retrieval power of machines, with the close-reading skills of the historian. KWIC (or concordance) centres a corpus on a specific query term, with `n` words (or characters) to the left and the right. 

In this section, we investigate reports of the London Medical Officers of Health, the [London's Pulse corpus](https://wellcomelibrary.org/moh/). 

> The reports were produced each year by the Medical Officer of Health (MOH) of a district and set out the work done by his public health and sanitary officers. The reports provided vital data on birth and death rates, infant mortality, incidence of infectious and other diseases, and a general statement on the health of the population. 

Source: https://wellcomelibrary.org/moh/about-the-reports/about-the-medical-officer-of-health-reports/

**[Important]** Before you continue run the cell to downlaod and extract the data we need in the exercises.

In [1]:
!mkdir working_data

mkdir: working_data: File exists


In [2]:
!wget -O working_data/python.zip https://github.com/kasparvonbeelen/ghi_python/blob/main/data/MOH/python.zip?raw=true
!unzip  -o working_data/python.zip -d working_data/moh

--2021-06-16 14:39:09--  https://github.com/kasparvonbeelen/ghi_python/blob/main/data/MOH/python.zip?raw=true
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/kasparvonbeelen/ghi_python/raw/main/data/MOH/python.zip [following]
--2021-06-16 14:39:09--  https://github.com/kasparvonbeelen/ghi_python/raw/main/data/MOH/python.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kasparvonbeelen/ghi_python/main/data/MOH/python.zip [following]
--2021-06-16 14:39:10--  https://raw.githubusercontent.com/kasparvonbeelen/ghi_python/main/data/MOH/python.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:

  inflating: working_data/moh/python/PoplarMetropolitanBorough.1929.b18246011.txt  
  inflating: working_data/moh/python/CityofWestminster.1958.b18248202.txt  
  inflating: working_data/moh/python/CityofWestminster.1937.b18247994.txt  
  inflating: working_data/moh/python/PoplarMetropolitanBorough.1921.b18245936.txt  
  inflating: working_data/moh/python/CityofWestminster.1938.b18248007.txt  
  inflating: working_data/moh/python/CityofWestminster.1947.b18248093.txt  
  inflating: working_data/moh/python/PoplarMetropolitanBorough.1950.b18246229.txt  
  inflating: working_data/moh/python/Westminster.1891.b2005709x.txt  
  inflating: working_data/moh/python/Westminster.1857.b18248342.txt  
  inflating: working_data/moh/python/CityofWestminster.1933.b18247957.txt  
  inflating: working_data/moh/python/Poplar.1899.b18222894.txt  
  inflating: working_data/moh/python/CityofWestminster.1944.b18248068.txt  
  inflating: working_data/moh/python/CityofWestminster.1909.b1824774x.txt  
  inflating

We start by importing the necessary libraries. Some of the code is explained in previous Notebooks, so won't discuss it in detail here.

The tools we need are:
- `nltk`: Natural Language Toolkint: for tokenization and concordance
- `pathlib`: a library for managing files and folders

In [5]:
import nltk # import natural language toolkit
from pathlib import Path # import Path object from pathlib
from nltk.tokenize import wordpunct_tokenize # import word_tokenize function from nltk.tokenize

In [6]:
!ls working_data/moh/python/ # list all files in data/MOH/python/

CityofWestminster.1901.b18247660.txt
CityofWestminster.1902.b18247672.txt
CityofWestminster.1903.b18247684.txt
CityofWestminster.1904.b18247696.txt
CityofWestminster.1905.b18247702.txt
CityofWestminster.1906.b18247714.txt
CityofWestminster.1907.b18247726.txt
CityofWestminster.1908.b18247738.txt
CityofWestminster.1909.b1824774x.txt
CityofWestminster.1910.b18247751.txt
CityofWestminster.1911.b18247763.txt
CityofWestminster.1912.b18247775.txt
CityofWestminster.1913.b18247787.txt
CityofWestminster.1914.b18247799.txt
CityofWestminster.1915.b18247805.txt
CityofWestminster.1917.b18247817.txt
CityofWestminster.1920.b18247829.txt
CityofWestminster.1921.b18247830.txt
CityofWestminster.1922.b18247842.txt
CityofWestminster.1923.b18247854.txt
CityofWestminster.1924.b18247866.txt
CityofWestminster.1925.b18247878.txt
CityofWestminster.1926.b1824788x.txt
CityofWestminster.1927.b18247891.txt
CityofWestminster.1928.b18247908.txt
CityofWestminster.1929.b1824791x.txt
CityofWestminster.1930.b18247921.txt
C

The data are stored in the following folder structure:

```
working_data
|___ moh
     |___ python
          |____ CityofWestminster.1901.b18247660.txt
          |____ ...
```

The code below:
- harvests all paths to `.txt` files in `working_data/moh/python`
- converts the result to a `list`

In [7]:
moh_reports_paths = list(Path('working_data/moh/python').glob('*.txt')) # get all txt files in data/MOH/python

We can print the paths to ten document with list slicing: `[:10]` means, get document from index positions `0` till `9`. (i.e. the first ten items).

In [8]:
print(moh_reports_paths[:10]) # print the first ten items

[PosixPath('working_data/moh/python/PoplarMetropolitanBorough.1945.b18246175.txt'), PosixPath('working_data/moh/python/CityofWestminster.1932.b18247945.txt'), PosixPath('working_data/moh/python/CityofWestminster.1921.b18247830.txt'), PosixPath('working_data/moh/python/PoplarandBromley.1900.b18245754.txt'), PosixPath('working_data/moh/python/Poplar.1919.b18120878.txt'), PosixPath('working_data/moh/python/PoplarMetropolitanBorough.1920.b18245924.txt'), PosixPath('working_data/moh/python/CityofWestminster.1907.b18247726.txt'), PosixPath('working_data/moh/python/CityofWestminster.1906.b18247714.txt'), PosixPath('working_data/moh/python/CityofWestminster.1903.b18247684.txt'), PosixPath('working_data/moh/python/PoplarMetropolitanBorough.1902.b18245778.txt')]


Once we know where all the files are located we can create a corpus.

To do this, we apply the following steps:

- create an empty list variable where we will store the tokens of the corpus (line 3)
- iterate over the collected paths (line 5)
- read the text file (line 6)
- lowercase the text (line 6)
- tokenize the string (line 7): this converts the string to a list of tokens
- iterate over tokens (line 8)
- test if a token contains only alphabetic characters (line 9)
- add a token to the list if line 9 evaluates to True (line 10)

The general flow of the program is similar to what we've seen before: we create an empty list where we store information from our text collection, in this case, all alphabetic tokens.

We use one more Notebook functionality `%%time` to print how long the cell took to run.

It could take a few seconds for the cell to run, so please be a bit patient:

In [10]:
%%time

corpus = [] # inititialize an empty list where we will store the MOH reports

for p in moh_reports_paths: # iterate over the paths to MOH reports, p will take the value of each item in moh_reports_paths 
    text_lower = open(p).read().lower() # read the text files and lowercase the string
    tokens = wordpunct_tokenize(text_lower) # tokenize the string
    for token in tokens: # iterate over the tokens
        if token.isalpha(): # test if token only contains alphabetic characteris
            corpus.append(token) # if the above test evaluates to True, append token to the corpus list
print('collected', len(corpus),'tokens')

collected 3550169 tokens
CPU times: user 2.82 s, sys: 161 ms, total: 2.98 s
Wall time: 3.08 s


While this program works perfectly fine, it's not the most efficient code. The example below is a bit better, especially if you're confronted with lots of text files. 

- the `with open` statement is a convenient way of handling the opening **and** closing of files (to make sure you don't keep all information in memory), which would slow down the execution of your program
- line 8 shows a list comprehension, this is similar to a `for` loop but faster and more concise.

We won't spend too much time discussing list comprehensions. The example below should suffice for now. We write a small program that collects odd numbers. First, we generate a list of numbers with `range(10)`...

In [11]:
# see the output of range(10)
list(range(10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

... we test for division by 2: `%` is the **modulus operator**: "which returns the remainder after dividing the left-hand operand by right-hand operand". `n % 2` evaluates to `0` if a number `n` can be divided by `2`. In Python `0` is equal to `False`, meaning if `n % 2` evaluates to `0`/`False` we won't append the number to `odd`. if it evaluates to any other integer, we'll append `n` to `odd`.

In [16]:
print(10%2)
print(15%2)

0
1


In [14]:
%%time
# program for find odd numbers
numbers = range(10) # get numbers 0 to 9
odd = [] # empty list where we store even numbers
for k in numbers: # iterate over numbers
    if k % 2: # test if number if divisible by 2
        odd.append(k) # if True append
print(odd) # print number of tokens collected

[1, 3, 5, 7, 9]
CPU times: user 213 µs, sys: 141 µs, total: 354 µs
Wall time: 286 µs


The same can be achieved with just one line of code using a list comprehension.

In [13]:
%time
odd = [k for k in range(10) if k % 2]
print(odd)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs
[1, 3, 5, 7, 9]


### -- Exercise

To see differences in performance, do the following:

- Remove the `print()` statement
- Increase the size of the list, i.e. change `range(10)` to `range(1000000)`.
- Compare the **Wall time** of these cells

Now returning to our example: run the slightly more efficient code and observe that it produces the same output, just faster!

In [18]:
%%time

corpus = [] # inititialize an empty list where we will store the MOH reports

for p in moh_reports_paths: # iterate over the paths to MOH reports, p will take the value of each item in moh_reports_paths 
    with open(p) as in_doc: # make sure to close the document after opening it
        tokens = wordpunct_tokenize(in_doc.read().lower())
        corpus.extend([t for t in tokens if t.isalpha()]) # list comprehension    
print('collected', len(corpus),'tokens') # print number of tokens collected

collected 3550169 tokens
CPU times: user 2.4 s, sys: 150 ms, total: 2.55 s
Wall time: 2.62 s


After collecting all tokens in a `list` we can convert this to another data type: an NLTK `Text` object. The cell below shows the results of the conversion.

In [20]:
print(type(corpus))
nltk_corpus = nltk.text.Text(corpus) # convert the list of tokens to a nltk.text.Text object
print(type(nltk_corpus))

<class 'list'>
<class 'nltk.text.Text'>


Why is this useful? Well the  NLTK`Text` object comes with many useful methods for corpus exploration. To inspect all the tools attached to a `Text` object, apply the `help()` function to `nltk_corpus` or (`help(nltk.text.Text)` does the same trick). You have to scroll down a bit (ignore all methods starting with `__`) to inspect the class methods.

In [21]:
help(nltk_corpus) # show methods attached to the nltk.text.Text object or nltk_corpus variable

Help on Text in module nltk.text object:

class Text(builtins.object)
 |  Text(tokens, name=None)
 |  
 |  A wrapper around a sequence of simple (string) tokens, which is
 |  intended to support initial exploration of texts (via the
 |  interactive console).  Its methods perform a variety of analyses
 |  on the text's contexts (e.g., counting, concordancing, collocation
 |  discovery), and display the results.  If you wish to write a
 |  program which makes use of these analyses, then you should bypass
 |  the ``Text`` class, and use the appropriate analysis function or
 |  class directly instead.
 |  
 |  A ``Text`` is typically initialized from a given document or
 |  corpus.  E.g.:
 |  
 |  >>> import nltk.corpus
 |  >>> from nltk.text import Text
 |  >>> moby = Text(nltk.corpus.gutenberg.words('melville-moby_dick.txt'))
 |  
 |  Methods defined here:
 |  
 |  __getitem__(self, i)
 |  
 |  __init__(self, tokens, name=None)
 |      Create a Text object.
 |      
 |      :param tokens

Let's have a closer look at `.concordance()`. According to the official documentation this method 
> Prints a concordance for ``word`` with the specified context window. Word matching is not case-sensitive.

It takes multiple arguments:
    - word: query term
    - width: the context window, i.e. determines the number of character printed 
    - lines: determines the number of lines to show (i.e. KWIC examples)

The first line of the output states the total number of hits for the query term (`Displaying * of * matches:`)

The example code below prints the context of the word **"poor"**.

In [22]:
nltk_corpus.concordance('poor',width=100,lines=10) # print the context of poor, window = 100 character

Displaying 10 of 1112 matches:
mes difficult to arrange but the friends of the poor and the charity organisation society can often 
l analysis in one case the milk proved to be of poor quality the work is carried out in the ordinary
 fat fair quality between per cent and per cent poor quality between per cent and per cent adulterat
able i district total good quality fair quality poor quality adulterated no percent no percent no pe
in which the applicant is already in receipt of poor law relief or is considered ought to be referen
ing cases previously notified under to to total poor law institutions sanatoria poor law institution
der to to total poor law institutions sanatoria poor law institutions sanatoria pulmonary males fema
g to tuberculosis and the treatment of cases in poor law and other hospitals advance in social well 
y in which the fat was between and per cent and poor or inferior quality in which the fat was betwee
ood quality no per cent fair quality no percent poor quality

### --Exercise

Use KWIC analysis to compare the word "poor" in MOsH reportss from the City of Westminster and Poplar. Using everything you learned the previous Notebook
- Create two subcopora one with Westminster, one with Poplar reports
- Tokenize the texts and convert the list of tokens to an NLTK `Text` object
- Use concardance to analyse the context of the work "poor"

In [29]:
# Enter code here

## 6.2 Collocations

While KWIC analysis is useful for investigating the context of words, it is a method that doesn't scale well: it helps with the close reading of around 100 words, but when examples run in the thousands it becomes more difficult. Collocations can help quantify the semantics of term, or how the meaning of words is different between corpora or subsamples of a corpus.

Collocations, as explained in the AntConc section, are often multi-word expressions containing tokens that tend to co-occur, such "New York City" (the span between words can be longer, they don't have to appear next to each other).

The NLTK `Text` object has `collocations()` function. Below we print and explain the documentation.

> collocations(self, num=20, window_size=2)
    Print collocations derived from the text, ignoring stopwords.
    
It has the following parameters:
> `:param num:` The maximum number of collocations to print.

The number of collocations to print (if not specified it will print 20)

> `:param window_size:` The number of tokens spanned by a collocation (default=2)

If `window_size=2` collocations will only include bigrams (words occurring next to each other). But sometimes we wish to include longer intervals, to make the co-occurrence of words within a broader window more visible, this allows us to go beyond multiword expressions and study the distribution of words in a corpus more generally. For example, we could look if "men" and "women" are discussed in each other's context (within a span of 10), even if they don't appear next to each other. 

In [40]:
%%time
nltk_corpus.collocations(window_size=2)

per cent; public health; county council; london county; medical
officer; scarlet fever; whooping cough; males females; local
government; legal proceedings; dwelling houses; poplar bromley; small
pox; ice cream; sub district; government board; child welfare; city
council; death rate; bromley bow
CPU times: user 9.35 s, sys: 51.4 ms, total: 9.4 s
Wall time: 9.45 s


In [28]:
%%time 
nltk_corpus.collocations(window_size=5)

street street; per cent; public health; county council; months months;
london county; medical officer; scarlet fever; males females; poplar
bromley; london council; road street; bromley bow; see page; road
road; whooping cough; officer health; medical health; poplar bow;
small pox
CPU times: user 27.2 s, sys: 260 ms, total: 27.4 s
Wall time: 27.6 s


While the `.collocations()` method provides a convenient tool for obtaining collocations from a corpus, its functionality remains rather limited. Below we will inspect the collocation functions of NLTK in more detail, giving you more power as well as precision.

Before we start we import all the required tools that `nltk.collocations` provides. This is handled by the `import *`, similar to a wildcard, it matches and loads all functions from `nltk.collocations`.

In [30]:
import nltk
from nltk.collocations import *

We have to select an association measure to compute the "strength" with which two tokens are "attracted" to each other. In general, collocations are words that are likely to appear together (within a specific context or window size). This explains why "the "red wine" is a strong collocation and "the wine" less so.

NLTK provides us with different measures, which you can print and investigate in more detail. Many of the functions refer to the classic NLP Handbook of Manning and Schütze, ["Foundations of statistical natural language processing"](https://nlp.stanford.edu/fsnlp/).

In [34]:
bigram_measures = nltk.collocations.BigramAssocMeasures()

In [35]:
help(bigram_measures)

Help on BigramAssocMeasures in module nltk.metrics.association object:

class BigramAssocMeasures(NgramAssocMeasures)
 |  A collection of bigram association measures. Each association measure
 |  is provided as a function with three arguments::
 |  
 |      bigram_score_fn(n_ii, (n_ix, n_xi), n_xx)
 |  
 |  The arguments constitute the marginals of a contingency table, counting
 |  the occurrences of particular events in a corpus. The letter i in the
 |  suffix refers to the appearance of the word in question, while x indicates
 |  the appearance of any word. Thus, for example:
 |  
 |      n_ii counts (w1, w2), i.e. the bigram being scored
 |      n_ix counts (w1, *)
 |      n_xi counts (*, w2)
 |      n_xx counts (*, *), i.e. any bigram
 |  
 |  This may be shown with respect to a contingency table::
 |  
 |              w1    ~w1
 |           ------ ------
 |       w2 | n_ii | n_oi | = n_xi
 |           ------ ------
 |      ~w2 | n_io | n_oo |
 |           ------ ------
 |         

In our example we use pointwise mutual inforamtion (pmi) to compute collocations.

In [38]:
help(bigram_measures.pmi)

Help on method pmi in module nltk.metrics.association:

pmi(*marginals) method of abc.ABCMeta instance
    Scores ngrams by pointwise mutual information, as in Manning and
    Schutze 5.4.





![pmi](https://miro.medium.com/max/930/1*OoI8_cZQwYGJEUjzozBOCw.png)

`pmi` is a rather straightforward metric, in the case of bigrams (i.e. collocations of length two and window size two):
- compute the total number of tokens in a corpus, assume this is `n` (3435)
- compute the probability of  `a` and `b` appearing as a bigram. If the bigram `(a,b)` occurs 10 times, the probability (`P(a,b)` = 10/3435 = 0.0029)
- compute the probability of observing `a` and `b` across the whole corpus. For example if `a` appears `30` times and b `45`, their respective probabilities are `P(a)` = 30/3435 = 0.0087 and P(b) = 45/3435 = 0.0131. We then multiple `P(a)` and `P(b)` to obtain the denominator 0.0087 `*` 0.0131 = 0.0001
- next we 0.0029 / 0.0001 = 28.9999 and log this value log2(28.9999)

In [58]:
from numpy import log2
nom = 10/3435
denom = (30/3435) * (45/3435)
mpi = log2(nom/denom)
mpi

4.6692787866546315

To get collocations by their `pmi` scores, we apply the `.from_words()` method to the `nltk_corpus` (or any list of tokens). The result of this operation is stored in a `finder` object which we can subsequently used to rank and print collocations. 

Note that the results below look somewhat strange, these aren't very meaningful collocates.

In [59]:
finder = BigramCollocationFinder.from_words(nltk_corpus)
finder.nbest(bigram_measures.pmi, 10) 

[('abso', 'lutely'),
 ('acidi', 'lacfc'),
 ('acquires', 'setiological'),
 ('adolph', 'mussi'),
 ('adolphus', 'massie'),
 ('adultorated', 'sanples'),
 ('adver', 'tising'),
 ('aeql', 'rrhage'),
 ('alathilde', 'christoffersen'),
 ('alio', 'wances')]

These results are rather spurious, why? If, for example `a` and `b` both appear only once **and** next to each other, the `pmi` score will be high. But such collocations aren't meaningful collocation, more a rare artefact of the data.
To solve this problem, we filter by ngram frequency, removing in our case all bigrams that appear less than 3 times with `.apply_freq_filter()` function.

In [52]:
help(finder.apply_freq_filter)

Help on method apply_freq_filter in module nltk.collocations:

apply_freq_filter(min_freq) method of nltk.collocations.BigramCollocationFinder instance
    Removes candidate ngrams which have frequency less than min_freq.



In [53]:
finder.apply_freq_filter(3)
finder.nbest(bigram_measures.pmi, 10)

[('bowers', 'gifford'),
 ('carrie', 'simuelson'),
 ('culex', 'pipiens'),
 ('heatherfield', 'ascot'),
 ('holmes', 'godson'),
 ('lehman', 'ashmead'),
 ('locum', 'tenens'),
 ('nemine', 'contradicente'),
 ('quinton', 'polyclinic'),
 ('rhesus', 'incompatibility')]

Now many names appear. We can even be more strict and use a higher threshold for filtering.

In [55]:
finder.apply_freq_filter(20)
finder.nbest(bigram_measures.pmi, 10)

[('braxton', 'hicks'),
 ('herman', 'olsen'),
 ('posterior', 'basal'),
 ('arterio', 'sclerosis'),
 ('brucella', 'abortus'),
 ('burnishers', 'diamond'),
 ('pillows', 'bolsters'),
 ('carvers', 'gilders'),
 ('sweetmeats', 'cosaques'),
 ('bookbinding', 'lithographers')]

It is also possible to change the window size, but the larger the window size the longer the computation takes

In [ ]:
finder = BigramCollocationFinder.from_words(nltk_corpus, window_size = 5)
finder.apply_freq_filter(10)
finder.nbest(bigram_measures.pmi, 10)

Lastly, you can focus on collocations that contains a specific token, i.e. for example get all collocations with the token "poor". We have pass function to `.apply_ngram_filter()`. At this point, you shouldn't worry about the code, only understand how to adapt it (see exercise below). 

In [61]:
def token_filter_poor(*w):
     return 'poor' not in w

finder = BigramCollocationFinder.from_words(nltk_corpus)
finder.apply_freq_filter(3)
finder.apply_ngram_filter(token_filter_poor)
finder.nbest(bigram_measures.pmi, 10)

[('poor', 'attenders'),
 ('poor', 'palatines'),
 ('poor', 'genl'),
 ('poor', 'law'),
 ('compositions', 'poor'),
 ('poor', 'sufferers'),
 ('poor', 'quality'),
 ('sleep', 'poor'),
 ('poor', 'visibility'),
 ('failures', 'poor')]

### -- Exercise
Copy-paste the above code and create a program that prints the first 10 collocations with the word "women".
- change the frequency threshold
- explore otherr association measure, to what extent do your results change?

In [65]:
# Enter code here

### 6.3 Feature selection

In the last section of this Notebook, we explore computational methods for finding words that characterize a collection: we try to select tokens (more generally features) that distinguish a particular set of documents vis-a-vis another corpus. 

Such comparisons help us determine what type of language use was distinctive for a particular group or (such as a political party) period or location. We continue with the example of the MOsH reports, but compare the language of different boroughs, the affluent Westminster with the industrial, and considerably poorer, Poplar.

The code below should look familiar, but we made a few changes.\
- to make sure all data are in the right place, we download and extract it again
- we create two empty lists `corpus` and `labels`. In the former we store our text documents (each item in the list is one text file/string), the latter contains labels, `0` for Poplar and `1` for Westminster. We collect these labels in parallel with the text, i.e. the if the first item in `corpus` is a text from Westminster, the first label in `labels` is `1`.
- we use `with open` to automatically close each document after opening it (line 1)
- lines 9 - 12 contain an `if else` statement: if the string `westminster` appears in the file name we add `1` to `labels`, otherwise `0`.

In [86]:
!mkdir working_data
!wget -O working_data/python.zip https://github.com/kasparvonbeelen/ghi_python/blob/main/data/MOH/python.zip?raw=true
!unzip -o working_data/python.zip -d  working_data/moh

mkdir: working_data: File exists
--2021-06-16 16:56:17--  https://github.com/kasparvonbeelen/ghi_python/blob/main/data/MOH/python.zip?raw=true
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/kasparvonbeelen/ghi_python/raw/main/data/MOH/python.zip [following]
--2021-06-16 16:56:17--  https://github.com/kasparvonbeelen/ghi_python/raw/main/data/MOH/python.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kasparvonbeelen/ghi_python/main/data/MOH/python.zip [following]
--2021-06-16 16:56:18--  https://raw.githubusercontent.com/kasparvonbeelen/ghi_python/main/data/MOH/python.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubu

  inflating: working_data/moh/python/PoplarMetropolitanBorough.1933.b18246059.txt  
  inflating: working_data/moh/python/PoplarMetropolitanBorough.1924.b18245961.txt  
  inflating: working_data/moh/python/PoplarMetropolitanBorough.1954.b18246266.txt  
  inflating: working_data/moh/python/CityofWestminster.1930.b18247921.txt  
  inflating: working_data/moh/python/CityofWestminster.1962.b1824824x.txt  
  inflating: working_data/moh/python/CityofWestminster.1923.b18247854.txt  
  inflating: working_data/moh/python/PoplarMetropolitanBorough.1929.b18246011.txt  
  inflating: working_data/moh/python/CityofWestminster.1958.b18248202.txt  
  inflating: working_data/moh/python/CityofWestminster.1937.b18247994.txt  
  inflating: working_data/moh/python/PoplarMetropolitanBorough.1921.b18245936.txt  
  inflating: working_data/moh/python/CityofWestminster.1938.b18248007.txt  
  inflating: working_data/moh/python/CityofWestminster.1947.b18248093.txt  
  inflating: working_data/moh/python/PoplarMetro

In [79]:
%%time
import nltk # import natural language toolkit
from pathlib import Path # import Path object from pathlib
from nltk.tokenize import wordpunct_tokenize # import word_tokenize function from nltk.tokenize

moh_reports_paths = list(Path('working_data/moh/python').glob('*.txt')) # get all txt files in data/MOH/python

corpus = [] # save corpus here
labels = [] # save labels here

for r in moh_reports_paths: # iterate over documents
    with open(r) as in_doc: # open document (also take care close it later)
        corpus.append(in_doc.read().lower()) # append the lowercased document to corpus
        
        if 'westminster' in r.name.lower(): # check if westeminster appear in the file name
            labels.append(1) # if so, append 1 to labels
        else: # if not
            labels.append(0) # append 0 to labels

CPU times: user 223 ms, sys: 46 ms, total: 269 ms
Wall time: 270 ms


Each document should correspond to one label. The lists `labels` and `corpus` should have equal length.

In [81]:
print(len(labels),len(corpus))

159 159


In [82]:
print(len(labels) == len(corpus))

True


As said earlier, we collect labels for each document, `1` for Westminster and `0` for Poplar (it could also be reverse, of course!). It is important that each label corresponds correctly with a text file in `corpus`. 

In [83]:
print(labels[:10])

[0, 1, 1, 0, 0, 0, 1, 1, 1, 0]


We can check this by printing the first hundred characters of the first document (labelled as `0`)...

Note that `corpus[0]` returns the first document, from which we slice the first hundred character `[:100]`.

In [84]:
corpus[0][:100]

"l-'&rary pop s-/ metropolitan borough of poplar . abridged interim report on the health of the borou"

... and the second document (labelled as `0`)

In [87]:
corpus[1][:100]

'city of westminster. report of the medical officer of health for the year . - 1932 andrew j. shinnie'

Checking your code by eyeballing the output is always good practice. Even if your code runs, it could still contain bugs, which are commonly referred to as "semantic errors".

To obtain the most distinctive words (for both report from Westminster and Poplar) we use an external library [`TextFeatureSelection`](https://pypi.org/project/TextFeatureSelection/). Python has a very rich and fast-evolving ecosystem. If you have a problem, it's very likely someone wrote a library to help you with this problem. We first have to install this package (it's not yet part of Colab)

In [ ]:
!pip install TextFeatureSelection

Now we can apply the `TextFeatureSelection` library. The documentation is available [here](https://pypi.org/project/TextFeatureSelection/).

Computing the features requires only a few lines of code. You only need to provide 
- a corpus for the `input_doc_list` parameter
- a list of labels for the `target` parameter

`TextFeatureSelection` then uses various metrics to compute the extent to which words are associated with a label. The output of this process is a `pandas.DataFrame`. Working with tabular data and data frames will be extensively discussed in Part II of this course. For now, we show you how to sort information and get the most distinctive words or features.

In [91]:
help(TextFeatureSelection)

Help on class TextFeatureSelection in module TextFeatureSelection:

class TextFeatureSelection(builtins.object)
 |  TextFeatureSelection(target, input_doc_list, stop_words=None, metric_list=['MI', 'CHI', 'PD', 'IG'])
 |  
 |  Compute score for each word to identify and select words which result in better model performance.
 |  
 |      Parameters
 |      ----------
 |      target : list object which has categories of labels. for more than one category, no need to dummy code and instead provide label encoded values as list object.
 |  
 |      input_doc_list : List object which has text. each element of list is text corpus. No need to tokenize, as text will be tokenized in the module while processing. target and input_doc_list should have same length.
 |  
 |      stop_words : Words for which you will not want to have metric values calculated. Default is blank.
 |  
 |      metric_list : List object which has the metric to be calculated. There are 4 metric which are being computed as 'M

In [92]:
from TextFeatureSelection import TextFeatureSelection # import TextFeatureSelection
fsOBJ=TextFeatureSelection(target=labels,input_doc_list=corpus) # compute features
df=fsOBJ.getScore() # get features as a dataframe
df

,word list,word occurence count,Proportional Difference,Mutual Information,Chi Square,Information Gain
0,00,103,-0.009709,0.094959,2.463282,0.004326
1,000,149,0.073826,0.008605,0.150191,0.000266
2,000000,1,-1.000000,0.778445,1.185538,0.001507
3,0001,3,1.000000,-inf,2.595483,0.000000
4,000163,1,1.000000,-inf,0.854210,0.000000
...,...,...,...,...,...,...
42232,¾gallons,1,-1.000000,0.778445,1.185538,0.001507
42233,¾ths,1,-1.000000,0.778445,1.185538,0.001507
42234,ægis,1,1.000000,-inf,0.854210,0.000000
42235,æration,1,-1.000000,0.778445,1.185538,0.001507


A `pandas.DataFrame` is similar to an Excel speadsheet. It contain several columns which we can use for selecting and sorting information. In fact, if you are familiar with Excel, you can export the data frame and open it as a spreadsheet. The code below takes care of this.


In [90]:
result_df.to_excel('working_data/result_features.xlsx')

We use the following columns to select and rank words:
- **Word occurence count**: How often a term occurs in the corpus
- **Proportional Difference**: It helps ﬁnd unigrams that occur mostly in one class of documents or the other."
- **Mutual Information**: The discriminatory power of a word.

In [102]:
westminster_df = df[(df['word occurence count'] > 20 ) & (df['Proportional Difference'] > 0 )]
westminster_df.sort_values('Information Gain',ascending=False)[:10]

,word list,word occurence count,Proportional Difference,Mutual Information,Chi Square,Information Gain
21070,horseferry,71,0.971831,-3.484235,102.313762,0.239720
41176,wes,64,0.968750,-3.380438,84.840438,0.205713
30188,pimlico,63,0.968254,-3.364690,82.552451,0.201071
7989,await,64,0.906250,-2.281826,73.305433,0.165213
9838,buckingham,61,0.901639,-2.233817,66.974989,0.152432
20355,harrison,63,0.873016,-1.978396,65.767627,0.144732
33224,restaurant,58,0.896552,-2.183386,61.025147,0.140108
18111,fines,91,0.692308,-1.093357,79.850990,0.139906
10445,carpentry,47,0.957447,-3.071703,51.509404,0.133109
25739,marshall,46,0.956522,-3.050197,49.861808,0.129219


In [101]:
poplar_df = df[(df['word occurence count'] > 20 ) & (df['Proportional Difference'] < 0 )]
poplar_df.sort_values('Information Gain',ascending=False)[:10]

,word list,word occurence count,Proportional Difference,Mutual Information,Chi Square,Information Gain
30606,pop,59,-1.000000,0.778445,110.515890,0.184282
15282,dock,66,-0.787879,0.666327,85.911216,0.149069
22759,intimations,92,-0.478261,0.476164,68.933936,0.146510
22037,india,67,-0.761194,0.651290,82.833472,0.144441
31245,procured,70,-0.714286,0.624294,79.780218,0.141947
41149,wellington,86,-0.511628,0.498485,66.399455,0.131704
17897,ferry,68,-0.705882,0.619380,74.205624,0.129299
34547,seamen,65,-0.723077,0.629409,71.698892,0.122070
14529,devons,47,-1.000000,0.778445,78.605431,0.118591
26460,millwall,49,-0.959184,0.757825,77.262501,0.118218


In [100]:
poplar_df = df[(df['word occurence count'] > 20 ) & (df['Proportional Difference'] < 0 )]
poplar_df.sort_values('Chi Square',ascending=False)[:10]

,word list,word occurence count,Proportional Difference,Mutual Information,Chi Square,Information Gain
30606,pop,59,-1.000000,0.778445,110.515890,0.184282
9432,bow,89,-0.640449,0.580268,106.152339,0.000000
42219,zymotic,94,-0.553191,0.525609,93.326942,0.000000
15282,dock,66,-0.787879,0.666327,85.911216,0.149069
22037,india,67,-0.761194,0.651290,82.833472,0.144441
31245,procured,70,-0.714286,0.624294,79.780218,0.141947
14529,devons,47,-1.000000,0.778445,78.605431,0.118591
26460,millwall,49,-0.959184,0.757825,77.262501,0.118218
33936,ruston,46,-1.000000,0.778445,76.252152,0.114306
17897,ferry,68,-0.705882,0.619380,74.205624,0.129299


## Fin.